# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Откроем файл с данными и изучим их, для этого подключим библиотеку `pandas` помимо неё подключим еще остальные библиотеки, которые нам пригодятся. Для того чтобы прочитать данные из датасета воспользуемся методом `read_csv`, для получения общей информации о датасете воспользуемся методом `info`. Но для начала установим недостающие библиотеки.

In [1]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install catboost

In [3]:
!pip install swifter

In [4]:
import pandas as pd
import re
import nltk
from pymystem3 import Mystem
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings("ignore")
import swifter

Библиотеки импортировали, теперь инициализируем лемматизатор. Так как мы будем обучать модели машинного обучения на тексте, то необходимо убрать из датасета стоп-слова, которые не несут смысловой нагрузки, список стоп-слов тоже инициализируем. Также инициализируем необходимые данные для использования лемматизатора. 

In [5]:
#LEMM_STEM = Mystem()
LEMM_STEM = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
STOP_WORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DoctorLector\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DoctorLector\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DoctorLector\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DoctorLector\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DoctorLector\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Лемматизатор успешно инициализировался, можно переходить к изучению датасета:

In [6]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Как видно на данных индекс был вынесен в отдельный столбец `Unnamed: 0`, удалим данный столбец из данных, так как он лишний и непонятно в каком порядке там указаны индексы:

In [7]:
data = data.drop(columns=data.columns[0], axis=1)
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Лишний столбец удалили, посмотрим теперь информацию о датасете:

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


В датасете всего два столбца, с текстовыми комментариями и признаком о том является комментарий токсичным или нет. Также по данным видно, что пропусков в данных нет, проверим нет ли явных дубликатов:

In [9]:
data.duplicated().sum()

0

Явных дубликатов нет, проверим теперь есть ли неявные дубликаты в столбце `text`, возможно одинаковые комментарии указали со словами в разных регистрах. Переведем все комментари в нижний регистр:

In [10]:
data['text'] = data['text'].str.lower()

Посчитам теперь количество дубликатов:

In [11]:
data.duplicated().sum()

45

Получается у нас в датасете **45** одинаковых комментариев, удалим их:

In [12]:
data = data.drop_duplicates()

Проверим количество дубликатов:

In [13]:
data.duplicated().sum()

0

Дубликатов больше не осталось, перейдем теперь непосредственно к обработке текста, перед тем как преобразовывать тексты в векторы и обучать модели машинного обучения. Нам необходимо преоразовать все слова в комментариях в начальную форму, то есть получить их лемму, а также избавиться от всех лишних символов, кроме пробелов, так как пробелы разделяют слова между собой. Напишем отдельные функции, которые будет выполнять указанные действия, для получения лемм ~~будем использовать бибилиотеку `pymystem3`~~ будем использовать лемматизатор  `WordNetLemmatizer` из библиотеки `nltk`, для очистки текста `регулярные выражения`. Лемматизатор будем использовать с POS-тегом, чтобы он находил лемму с учетом контекста:

In [14]:
def clear_text(text):
    '''
    Функция удаляет из текста лишние символы
    '''
    new_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    new_text = " ".join(new_text.split())
    return new_text

In [15]:
def get_wordnet_pos(word):
    '''
    Функция возвращает POS-тэг по переданному слову
    '''
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    
    return tag_dict.get(tag, wordnet.NOUN)

In [16]:
def lemmatize(text):
    '''
    Функция преобразует текст, возращает леммы слов по переданному тексту
    '''
    #lemm_list = LEMM_STEM.lemmatize(text)
    lemm_list = [LEMM_STEM.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text) if w not in STOP_WORDS]
    lemm_text = " ".join(lemm_list).rstrip()
        
    return lemm_text

Функции написали, применим их теперь к комментариям в датасете, запишем преобразованный текст в отдельный столбец:

In [17]:
data['text_clear'] = data['text'].apply(lambda text_comment: clear_text(text_comment))
data

,text,toxic,text_clear
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...
...,...,...,...
159287,""":::::and for the second time of asking, when ...",0,and for the second time of asking when your vi...
159288,you should be ashamed of yourself \n\nthat is ...,0,you should be ashamed of yourself that is a ho...
159289,"spitzer \n\numm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...
159290,and it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...


In [19]:
data['text_lemm'] = data['text_clear'].swifter.progress_bar(True).apply(lambda text_comment: lemmatize(text_comment))
data

Pandas Apply:   0%|          | 0/159247 [00:00<?, ?it/s]

,text,toxic,text_clear,text_lemm
0,explanation\nwhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edits make username hardcore metal...
1,d'aww! he matches this background colour i'm s...,0,d aww he matches this background colour i m se...,aww match background colour seemingly stuck th...
2,"hey man, i'm really not trying to edit war. it...",0,hey man i m really not trying to edit war it s...,hey man really try edit war guy constantly rem...
3,"""\nmore\ni can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,make real suggestion improvement wonder sectio...
4,"you, sir, are my hero. any chance you remember...",0,you sir are my hero any chance you remember wh...,sir hero chance remember page
...,...,...,...,...
159287,""":::::and for the second time of asking, when ...",0,and for the second time of asking when your vi...,second time ask view completely contradicts co...
159288,you should be ashamed of yourself \n\nthat is ...,0,you should be ashamed of yourself that is a ho...,ashamed horrible thing put talk page
159289,"spitzer \n\numm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...,spitzer umm there actual article prostitution ...
159290,and it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...,look like actually put speedy first version de...


Текст успешно преобразовали, можем переходить к обучению моделей.

## Обучение

У нас в датасете сейчас 4 столбца, оставим только два столбца
- `text_lemm` — преобразованный и очищенный текст, будем этот столбец использовать в качестве признака;
- `toxic` — флаг о том, токсичный указанный комментарий или нет, этот столбец будем использовать в качестве целевого признака.

Удалим лишние данные из признаков оставив только выше указанные столбцы:

In [20]:
data_features = data.drop(['text', 'text_clear'], axis=1)

Лишние признаки удалили, теперь разделим признаки на обучающую и тестовую выборки, обучающую выборку возьмем равную **80%**, тестовую равную **20%**. Берем такое разделение, так как планируем для поиска моделей использовать кросс-валидацию и `GridSearchCV`, указанные методы сами внутри разделяют переданную выборку на обучающую и валидационную.

In [21]:
data_train, data_test = train_test_split(data_features, test_size=0.2, random_state=12345)
print(data_train.shape)
print(data_test.shape)

(127397, 2)
(31850, 2)


Как видно по размерности выборки разделились корректно, вынесем теперь признаки и целевой признак в отдельные переменные у обучающей и тестовой выборки:

In [22]:
data_train_features = data_train.drop(['toxic'], axis=1)
data_train_target = data_train['toxic']

data_test_features = data_test.drop(['toxic'], axis=1)
data_test_target = data_test['toxic']

Так как мы работаем с текстом, нам необходимо преобразовать этот текст в читаем вид для моделей машинного обучения, преобазуем текст в векторы с помощью *TF-IDF*, который учитывает важность слова в контексте документа для этого воспользуемся классов `TfidfVectorizer` из библиотеки `sklearn`:

In [23]:
count_tf_idf = TfidfVectorizer(stop_words=STOP_WORDS)
data_train_features_tf_idf = count_tf_idf.fit_transform(data_train_features['text_lemm'])
data_test_features_tf_idf = count_tf_idf.transform(data_test_features['text_lemm'])

Текст успешно преобразовался в векторы, теперь можно переходить к обучению моделей. Нам нужно определить по тексту указан положительный или негативный комментарий, то есть нужно решить задачу классификации, тогда для решения задачи будет использовать следующие модели — **Дерево решений**, **Случайный лес**, **Логистическая регрессия**. Для проверки моделей будем использовать метрику *F1*. Для поиска наилучшей модели сначала воспользуемся кросс-валидацией:

In [24]:
model_tree = DecisionTreeClassifier()
model_forest = RandomForestClassifier(n_jobs=-1)
model_clf = LogisticRegression(max_iter=1000)

scores_tree = cross_val_score(model_tree, data_train_features_tf_idf, data_train_target, scoring='f1', cv=3, n_jobs=-1)
scores_forest = cross_val_score(model_forest, data_train_features_tf_idf, data_train_target, scoring='f1', cv=3, n_jobs=-1)
scores_clf = cross_val_score(model_clf, data_train_features_tf_idf, data_train_target, scoring='f1', cv=3, n_jobs=-1)

print('F1 Кросс-валидация Дерево решений', abs(scores_tree.mean()))
print('F1 Кросс-валидация Случайного леса', abs(scores_forest.mean()))
print('F1 Кросс-валидация Логистической регрессии', abs(scores_clf.mean()))

F1 Кросс-валидация Дерево решений 0.7014340576052088
F1 Кросс-валидация Случайного леса 0.6762413603810483
F1 Кросс-валидация Логистической регрессии 0.7070644218247765


По заданию нам необходимо, чтобы значение метрики *F1* было не меньше **0.75**, с помощью кросс-валидации мы получили следующие резудьтаты по моделям:
- **Дерево решений** — *0.701*;
- **Случайный лес** — *0.676*;
- **Логистическая регрессия** — *0.707*.

Не одна модель не подходит под наши критерии, попробуем перебрать гиперпараметры у моделей **Дерево решений**, **Случайный лес** с помощью `GridSearchCV` может получиться подобрать модель с необходимым значением по метрике *F1*:

In [25]:
model_tree = DecisionTreeClassifier(random_state=12345)
params_grid_tree = {
    'max_depth': list(range(5, 61, 5))
}
grid_tree = GridSearchCV(model_tree, param_grid=params_grid_tree, scoring='f1', cv=3, n_jobs=-1)
grid_tree.fit(data_train_features_tf_idf, data_train_target)
print('Best params', grid_tree.best_params_)
print('Best score F1', abs(grid_tree.best_score_))

Best params {'max_depth': 60}
Best score F1 0.7026886079754443


При глубине деревьев равной **60** достигается наилучшее значение по метрике *F1* — **0.702**, но этого все равно недостаточно, по заданию значение метрики *F1* должно быть не меньше **0.75**. Попробуем теперь перебрать гиперпараметры у модели **Случайный лес**:

In [26]:
model_forest = RandomForestClassifier(random_state=12345, n_jobs=-1)
params_grid_forest = {
    'max_depth': list(range(5, 31, 5)),
    'n_estimators': list(range(10, 51, 10))
}
grid_forest = GridSearchCV(model_forest, param_grid=params_grid_forest, scoring='f1', cv=3, n_jobs=-1)
grid_forest.fit(data_train_features_tf_idf, data_train_target)
print('Best params', grid_forest.best_params_)
print('Best score F1', abs(grid_forest.best_score_))

Best params {'max_depth': 30, 'n_estimators': 10}
Best score F1 0.029615720103462802


При количестве деревьев равное **30** и глубине деревьев равной **10** у модели **Случайный лес** достигается наилучшее значение по метрике *F1* — **0.02** это слишком мало, наихудший результат по сравнению с другими моделями. Ни одна модель из выбранных ранее не показала нужное значение по метрике *F1*, воспользуемся моделью с градиентным спуском **CatBoost** для решения нашей задачи. Для перебора гиперпараметров для поиска наилучшей модели, будем использовать встроенный метод `grid_search`, чтобы метод `grid_search` при поиске модели использовал метрику *F1* укажем в атрибуте `eval_metric` значение *F1* при инициализации модели **CatBoost**. В модель **CatBoost** будем передавать текст не обработанный с помощью *TF-IDF*, так как **CatBoost** сам может преобразовать текстовые признаки:

In [27]:
model_catboost = CatBoostClassifier(random_state=12345, verbose=25, eval_metric='F1', iterations=100, 
                                    text_features=['text_lemm'])

In [28]:
params = {
    'max_depth': [5, 10],
    'learning_rate': [0.35, 0.45],
    'iterations': [100]
}
grid_catboost = model_catboost.grid_search(params, X=data_train[['text_lemm']], y=data_train[['toxic']], cv=3)
print('Best params', grid_catboost['params'])
print('Best score', sum(grid_catboost['cv_results']['test-F1-mean']) / len(grid_catboost['cv_results']['test-F1-mean']))

0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 166ms	remaining: 16.5s
25:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 329ms	remaining: 936ms
50:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 471ms	remaining: 453ms
75:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 627ms	remaining: 198ms
99:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 760ms	remaining: 0us

bestTest = 0
bestIteration = 0

0:	loss: 0.0000000	best: 0.0000000 (0)	total: 3.03s	remaining: 9.09s
0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 7.76ms	remaining: 768ms
25:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 150ms	remaining: 427ms
50:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 337ms	remaining: 324ms
75:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 468ms	remaining: 148ms
99:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 595ms	remaining: 0us

bestTest = 0
bestIteration = 0



При глубине дерева равной **5** и скорости обучения равной **0.35** модель показывает максимальный результат по метрике *F1* и он равен **0.75**, это наилучший результат по сравнению с другими моделями, также у этой модели подходящий показатель по метрике *F1* — не менее **0.75**. Будем использовать модель **CatBoost** для предсказаний на тестовой выборке, так как данная модель показала наилучший результат по сравнению с другими моделями. Обучим модель **CatBoost** с указанными гиперпараметрами и проверим её на тестовой выборке.

In [29]:
best_model_catboost = CatBoostClassifier(random_state=12345, verbose=25, eval_metric='F1', iterations=100, 
                                         text_features=['text_lemm'], depth=5, learning_rate=0.35)
best_model_catboost.fit(data_train[['text_lemm']], data_train[['toxic']])

0:	learn: 0.7081336	total: 54.7ms	remaining: 5.41s
25:	learn: 0.7366767	total: 1.7s	remaining: 4.84s
50:	learn: 0.7537963	total: 3.36s	remaining: 3.23s
75:	learn: 0.7601027	total: 4.93s	remaining: 1.56s
99:	learn: 0.7616096	total: 6.32s	remaining: 0us


Модель **CatBoost** успешно обучилась, проверим её теперь на тестовой выборке с помощью метрики *F1*:

In [30]:
best_model_catboost_predictions = best_model_catboost.predict(data_test[['text_lemm']])
print('F1 CatBoost на тестовой выборке:', f1_score(data_test_target, best_model_catboost_predictions))

F1 CatBoost на тестовой выборке: 0.7761702127659575


На тестовой выборке у модели **CatBoost** значение по метрике *F1* равно **0.77**, это выше **0.75**, что говорит о том, что мы сделали правильный выбор в пользу данной модели. Проверим теперь модель на адекватность с помощью `DummyClassifier`, которая будет равновероятно распределять предсказанные значения классов, в нашем случае положительные и негативные комментарии:

In [31]:
model_dummy = DummyClassifier(strategy='uniform')
model_dummy.fit(data_train_features_tf_idf, data_train_target)
predictions_dummy = model_dummy.predict(data_test_features_tf_idf)
result_dummy = f1_score(data_test_target, predictions_dummy)
print('F1 DummyClassifier на тестовой выборке', result_dummy)

F1 DummyClassifier на тестовой выборке 0.16910722720831367


Метрика *F1* на случайной модели имеет значение **0.17**, что значительно меньше у выбранной модели **CatBoost** и что говорит о том, что модель **CatBoost** делает адекватные предсказания.

## Общий вывод

В рамках задания нам необходимо было обучить модель, которая по тексту может классифицировать комментарий на позитивный и негативный. Для проверки моделей необходимо было использовать метрику *F1*. Изначально для обучения были выбраны следующие модели — **Дерево решений**, **Случайный лес**, **Логистическая регрессия**. Для обучения моделей использовалась кросс-валидация и `GridSearchCV`, были получены следующие результаты:

Кросс-валидация с метрикой *F1*:

- Дерево решений — **0.701**;
- Случайный лес — **0.676**;
- Логистическая регрессия — **0.707**.

`GridSearchCV` с метрикой *F1* лучшие показатели:

- Дерево решений — **0.702**;
- Случайный лес — **0.02**.

С помощью указанных выше методов не удалось найти модель со значением метрике *F1* не менее **0.75**, поэтому для обучения была выбрана модель с градиентным бустингом **CatBoost** с помощью метода `grid_search` удалось найти модель с подходящими гиперпараметрами и значением метрики *F1*. Данная модель была проверена на тестовой выборке и показала значение по метрике *F1* равное **0.77**, также данная модель была проверена на адекватность.

На основании выше изложенного для классификации комментариев на позитивные и негативные необходимо использовать модель **CatBoost**.